In [1]:
# importation des bibliothèques
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import cv2

In [2]:
# importer l'architecture et les poids du modèle de détection de visage
architecture = r"face_detector\deploy.prototxt"
poids = r"face_detector\weights.caffemodel"
faceModel = cv2.dnn.readNet(architecture, poids)

In [3]:
# importer le model de détection de mask
faceDetection = load_model(r"model\face_model.h5")

In [4]:
# initialiser le capteur de flux vidéo
vs = VideoStream(src=0).start()

In [5]:
def detect_face(frame, faceModel, faceDetection):
    # initialiser des listes pour les visages, leurs emplacements et les prédictions
    faces = []
    localisations = []
    predictions = []

    (h, l) = frame.shape[:2]
    # {h:hauteur , l:largeur}
    
    # prétraitement d'image
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300,300), mean=(104, 177, 123))

    # obtenir les détections de visage
    faceModel.setInput(blob)
    detection = faceModel.forward()

    for i in range(detection.shape[2]):
        confidence = detection[0, 0, i, 2] 
        if confidence > 0.5:
            #calculer les délimitations de la boîte
            boxes = detection[0, 0, i, 3:] * np.array([l,h, l,h])

            (gauche, haut, droite, bas) = boxes.astype('int')
            (gauche, haut) = (max(0, gauche)), max(0, haut)
            (droite, bas) = (min(l-1, droite), min(h-1, bas))

            face = frame[haut:bas , gauche:droite]

            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, dsize=(224, 224))
            face = img_to_array(face)      
            face = preprocess_input(face)  

            faces.append(face)
            localisations.append((gauche, haut, droite, bas))

    # nous faisons des prédictions si au moins un visage est détecté
    if len(faces) > 0:
        faces = np.array(faces, dtype='float32')
        predictions = faceDetection.predict(faces, batch_size=32)
    
    return(localisations, predictions)

In [6]:
while True:
    frame = vs.read()
    (locs, preds) = detect_face(frame, faceModel, faceDetection)

    for (box, pred) in zip (locs, preds):
        (mohamed, other, youssef) = pred
        if mohamed > youssef and mohamed > other:
            label = "mohamed"
            couleur = (0, 255, 0)
        elif youssef > mohamed and youssef > other:
            label = "youssef"
            couleur =(0, 255, 0)
        else:
            label = "Other"
            couleur = (0, 0, 255)

        pourcentage = "%.2f"%(max(mohamed, youssef, other) * 100)
        label = "{} : {}".format(label, pourcentage)

        (gauche, haut, droite, bas) = box

        # tracer un rectangle autour des visages
        cv2.rectangle(frame, (gauche, haut), (droite, bas), couleur, 2)
        # afficher la classe du visage
        cv2.putText(frame, label, (gauche, haut-15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, couleur, 2)

    cv2.imshow("Output", frame)    
    if cv2.waitKey(1) == ord('q'): 
        break
cv2.destroyAllWindows()
vs.stop()             

1/1 [==============================] - 0s 53ms/step
